# Computer Vision Model - Horse vs Human detector

This model will be constructed and trained to detect images of horses or humans.

## Get training data

In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /tmp/horse-or-human.zip

!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip \
    -O /tmp/validation-horse-or-human.zip

import os
import zipfile

zip_files = ['/tmp/horse-or-human.zip', '/tmp/validation-horse-or-human.zip']
zip_ref = None
for zip_file in zip_files:
    zip_ref = zipfile.ZipFile(zip_file, 'r')
    zip_ref.extractall(zip_file[:-4])
if zip_ref:
    zip_ref.close()

--2021-06-18 13:35:16--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.169.80, 172.217.16.240, 142.250.179.240, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.169.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘/tmp/horse-or-human.zip’

/tmp/horse-or-human 100%[===================>] 142.65M  12.3MB/s    in 11s     

2021-06-18 13:35:28 (12.6 MB/s) - ‘/tmp/horse-or-human.zip’ saved [149574867/149574867]

--2021-06-18 13:35:28--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.169.80, 172.217.16.240, 142.250.179.240, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.169.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 

## Define model and optimizer

In [5]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

print(model.summary())

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=1e-3),
              metrics=['acc'])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 98, 98, 32)        896       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 47, 47, 64)        18496     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 21, 21, 128)       73856     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 10, 10, 128)       0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 12800)            

## Organise data into generators

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# We can generate more traning data by augmenting our images using different techniques. This can help improve accuracy when there is little data available, however it can also lead to overfitting.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    #rotation_range=40,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    #shear_range=0.2,
    #zoom_range=0.2,
    #horizontal_flip=True,
    #fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    '/tmp/horse-or-human',
    target_size=(100, 100),
    batch_size=128,
    class_mode='binary'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
    '/tmp/validation-horse-or-human',
    target_size=(100, 100),
    class_mode='binary'
)

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


## Train model

In [10]:
history = model.fit(
    train_generator,
    steps_per_epoch=8,
    epochs=100,
    verbose=1,
    validation_data=validation_generator
)

Epoch 1/100
8/8 [==============================] - 15s 2s/step - loss: 1.5628 - acc: 0.5106 - val_loss: 0.6631 - val_acc: 0.5000
Epoch 2/100
8/8 [==============================] - 13s 2s/step - loss: 0.6694 - acc: 0.6340 - val_loss: 0.6532 - val_acc: 0.5117
Epoch 3/100
8/8 [==============================] - 16s 2s/step - loss: 0.6765 - acc: 0.7186 - val_loss: 1.2128 - val_acc: 0.6680
Epoch 4/100
8/8 [==============================] - 18s 2s/step - loss: 0.3910 - acc: 0.8521 - val_loss: 0.3993 - val_acc: 0.8633
Epoch 5/100
8/8 [==============================] - 14s 2s/step - loss: 0.3455 - acc: 0.8932 - val_loss: 1.0156 - val_acc: 0.8125
Epoch 6/100
8/8 [==============================] - 13s 2s/step - loss: 0.1744 - acc: 0.9444 - val_loss: 0.8880 - val_acc: 0.8359
Epoch 7/100
8/8 [==============================] - 13s 2s/step - loss: 0.0726 - acc: 0.9800 - val_loss: 1.4072 - val_acc: 0.8125
Epoch 8/100
8/8 [==============================] - 13s 2s/step - loss: 0.4337 - acc: 0.8877 - val

8/8 [==============================] - 13s 2s/step - loss: 3.0840e-06 - acc: 1.0000 - val_loss: 3.2247 - val_acc: 0.8633
Epoch 64/100
8/8 [==============================] - 13s 2s/step - loss: 2.0844e-06 - acc: 1.0000 - val_loss: 3.3394 - val_acc: 0.8711
Epoch 65/100
8/8 [==============================] - 13s 2s/step - loss: 7.1447e-07 - acc: 1.0000 - val_loss: 3.4138 - val_acc: 0.8672
Epoch 66/100
8/8 [==============================] - 13s 2s/step - loss: 5.9979e-07 - acc: 1.0000 - val_loss: 3.4241 - val_acc: 0.8711
Epoch 67/100
8/8 [==============================] - 13s 2s/step - loss: 1.1245 - acc: 0.9622 - val_loss: 22.6209 - val_acc: 0.5820
Epoch 68/100
8/8 [==============================] - 13s 2s/step - loss: 0.6258 - acc: 0.9455 - val_loss: 1.1024 - val_acc: 0.8477
Epoch 69/100
8/8 [==============================] - 13s 2s/step - loss: 0.0109 - acc: 0.9978 - val_loss: 1.1940 - val_acc: 0.8594
Epoch 70/100
8/8 [==============================] - 14s 2s/step - loss: 0.0036 - acc: 

## Run the model
Run the model by uploading an image of your choice. The output will be a prediction - a horse or a human.

In [20]:
from ipywidgets import FileUpload
upload = FileUpload()
display(upload)

FileUpload(value={}, description='Upload')

[]


In [35]:
import numpy as np
from keras.preprocessing import image
from PIL import Image
import io

for name, data in upload.value.items():
    raw_data = data['content']
    img = Image.open(io.BytesIO(raw_data))
    img = img.convert('RGB')
    img = img.resize((100, 100), Image.NEAREST)
    img = image.img_to_array(img)

    x = image.img_to_array(img)
    x = x / 255.0
    x = np.expand_dims(x, axis=0)

    image_tensor = np.vstack([x])
    classes = model.predict(image_tensor)
    if classes[0]>0.5:
        print(name + " is a human")
    else:
        print(name + " is a horse")

[[1.4459301e-20]]
[1.4459301e-20]
download.webp is a horse
